In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz
!mkdir -p data
!tar -xzf dev-clean.tar.gz
!mv LibriSpeech data/LibriSpeech

# !wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/8342a090-89e7-4402-961e-1851da11e1aa/download/qutnoise.zip
# !mv qutnoise.zip data/qutnoise.zip
# !cd data && unzip qutnoise.zip

!wget https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/9b0f10ed-e3f5-40e7-b503-73c2943abfb1/download/qutnoisecafe.zip
!mv qutnoisecafe.zip data/qutnoisecafe.zip
!cd data && unzip qutnoisecafe.zip

--2021-04-07 17:34:36--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337926286 (322M) [application/x-gzip]
Saving to: ‘dev-clean.tar.gz’

dev-clean.tar.gz    100%[===================>] 322.27M  29.2MB/s    in 12s     

2021-04-07 17:34:48 (27.5 MB/s) - ‘dev-clean.tar.gz’ saved [337926286/337926286]

--2021-04-07 17:34:52--  https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/9b0f10ed-e3f5-40e7-b503-73c2943abfb1/download/qutnoisecafe.zip
Resolving data.researchdatafinder.qut.edu.au (data.researchdatafinder.qut.edu.au)... 131.181.186.101
Connecting to data.researchdatafinder.qut.edu.au (data.researchdatafinder.qut.edu.au)|131.181.186.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1711572070 (1.6G) [application

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 5.4MB/s 


In [ ]:
import glob
import torchaudio
import csv
import librosa
import IPython.display as ipd
import soundfile as sf

In [ ]:
## CSV --> Path, Label
csv_file = open('metadata.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(["file_path", "label"])

17

In [ ]:
## List all librispeech files
files_libri = glob.glob('./data/LibriSpeech/dev-clean/**/*.flac', recursive=True)

In [ ]:
len(files_libri)

2703

## Splitting this numpy array of audio clip, whose sampling rate=16000, into clips of 5secs each. Thus each clip split should be a numpy array of size (5*16000 = 80000). So here I am basically taking windowed clips of [i:i+80000] -> set i=i+80000. This hopefully will give non-overlapping 5secs audio clips 

In [ ]:
!mkdir -p final_data/noise

In [ ]:
y, sr = librosa.load('./data/QUT-NOISE/QUT-NOISE/CAFE-CAFE-1.wav')
y_16k = librosa.resample(y, sr, 16000)

In [ ]:
sample_rate = 16000
time = 5
all_noise = []

i=0
cnt=0
while((i+(sample_rate*time))<len(y_16k)):
    x = y_16k[i:i+sample_rate*time]
    all_noise.append(x.reshape(1,-1))
    i = i+(sample_rate*time)
    f_name = f'./final_data/noise/CAFE-CAFE-1_trim_{time}s_{cnt}.wav'
    sf.write(f_name, x, sample_rate)
    writer.writerow([f_name, "0"])
    cnt += 1


In [ ]:
len(all_noise)

506

In [ ]:
ipd.Audio(data=all_noise[14], rate=16000)

In [ ]:
from tqdm import tqdm

In [ ]:
w=files_libri[0].split('/')[-1][:-5]
f_name = f'./final_data/speech/{w}.wav'

## Processing librispeech Data similarly. If clip more than 5s trim it. Else fill with empty sound (0 pad)


In [ ]:
files_libri[0]
!mkdir -p final_data/speech

In [ ]:

sample_rate = 16000
time = 5
all_libri = []


for libri_f in tqdm(files_libri):
    y, sr = librosa.load(libri_f)
    y_16k = librosa.resample(y, sr, 16000)
    
    if(len(y_16k)>=sample_rate*time):
        ## if clip more than 5s trim it
        x = y_16k[:sample_rate*time]
    else:
        ## if clip less than 5s pad with 0s
        x = y_16k
        x = np.pad(y_16k,(0,sample_rate*time-len(y_16k)), 'constant', constant_values=(0,0))

    all_libri.append(x.reshape(1,-1))
    i = i+(sample_rate*time)
    fn=libri_f.split('/')[-1][:-5]
    f_name = f'./final_data/speech/{fn}.wav'
    sf.write(f_name, x, sample_rate)
    writer.writerow([f_name, "1"])


100%|██████████| 2703/2703 [15:27<00:00,  2.92it/s]


In [ ]:
csv_file.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r final_data drive/MyDrive/ASR_Project_Shared

In [ ]:
!cp -r metadata.csv drive/MyDrive/ASR_Project_Shared

In [ ]:
ipd.Audio(tx, rate=16000)